# California's Vehicle Subsidy

In [1]:
from openfisca_us import Microsimulation, ACS
from ca_ubi.vehicle_subsidy import vehicle_subsidy
from ca_ubi.ubi import create_ubi_reform

baseline = Microsimulation(dataset=ACS, year=2019)
state = baseline.calc("state_code")
in_ca = state == "CA"
vehicle_subsidy_simulation = Microsimulation(vehicle_subsidy, dataset=ACS, year=2019)
cost_of_vehicle_subsidy = vehicle_subsidy_simulation.calc("vehicle_subsidy_payment", map_to="household")[in_ca].sum()
ubi_amount = cost_of_vehicle_subsidy / in_ca.sum()
ubi_simulation = Microsimulation(create_ubi_reform(ubi_amount), dataset=ACS, year=2019)

/home/nikhil/policyengine/openfisca-us/openfisca_us/variables/demographic/person/vehicles_owned.py:28: RuntimeWarning: invalid value encountered in remainder
  % num_adults_in_household


In [30]:
import pandas as pd
import plotly.express as px
from ubicenter import format_fig
from ubicenter.plotly import BLUE_COLOR_SEQUENCE

def get_poverty_rates(simulation: Microsimulation, label: str, deep_poverty: bool = False) -> pd.DataFrame:
    in_poverty = simulation.calc(f"spm_unit_is_in{'_deep' if deep_poverty else ''}_spm_poverty", map_to="person")
    group_names = ["Children", "Adults", "Seniors", "All"]
    poverty_rates = []
    for group in ("is_child", "is_wa_adult", "is_senior", "people"):
        poverty_rates.append(in_poverty[simulation.calc(group) > 0].mean())
    return pd.DataFrame({
        "Group": group_names,
        "Poverty rate": poverty_rates,
        "Label": label,
        "Type": "Poverty" if not deep_poverty else "Deep poverty",
    })

def get_regular_and_deep_poverty_rates(simulation: Microsimulation, label: str) -> pd.DataFrame:
    regular_df = get_poverty_rates(simulation, label, deep_poverty=False)
    deep_df = get_poverty_rates(simulation, label, deep_poverty=True)
    return pd.concat([regular_df, deep_df])

simulations = [baseline, vehicle_subsidy_simulation, ubi_simulation]
labels = ["Baseline", "Vehicle subsidy", "UBI"]

poverty_rate_df = pd.concat([get_regular_and_deep_poverty_rates(simulation, label) for simulation, label in zip(simulations, labels)])

hover_cards = []

for i, row in poverty_rate_df.iterrows():
    poverty_label = "poverty" if row["Type"] == "Poverty" else "deep poverty"
    if row["Label"] == "Baseline":
        hover = f"The current {poverty_label} rate for <b>{row['Group'].lower()}</b> is <b>{row['Poverty rate']:.1%}</b>"
    else:
        baseline_rate = poverty_rate_df.loc[(poverty_rate_df["Label"] == "Baseline") & (poverty_rate_df["Type"] == row["Type"]), "Poverty rate"].iloc[0]
        if row["Label"] == "Vehicle subsidy":
            # Something like: "Under the vehicle subsidy, the poverty rate for children would rise/fall to x% (+/- y%)"
            hover = f"Under the vehicle subsidy, the{poverty_label} rate for <b>{row['Group'].lower()}</b> would fall<br> to <b>{row['Poverty rate']:.1%}</b> ({(row['Poverty rate'] - baseline_rate) / baseline_rate:.1%})"
        else:
            # Something like: "Under a UBI, the poverty rate for children would rise/fall to x% (+/- y%)"
            hover = f"Under a UBI, the {poverty_label} rate for <b>{row['Group'].lower()}</b> would fall<br> to <b>{row['Poverty rate']:.1%}</b> ({(row['Poverty rate'] - baseline_rate) / baseline_rate:.1%})"
    hover_cards.append(hover)

poverty_rate_df["hover_card"] = hover_cards
fig = px.bar(
    poverty_rate_df, 
    x="Group", 
    y="Poverty rate", 
    color="Label", 
    barmode="group", 
    color_discrete_sequence=BLUE_COLOR_SEQUENCE, 
    custom_data=["hover_card"],
    animation_frame="Type",
).update_layout(
    title="Poverty rate by age group",
    yaxis_tickformat=".0%",
    legend_title=""
).update_traces(
    hovertemplate="%{customdata[0]}",
)
format_fig(fig)